<a href="https://colab.research.google.com/github/mikailajhay/South-Asia-E-Commerce-Analysis/blob/main/South_Asia_E_Commerce_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install plotly

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [3]:
df = pd.read_csv("E-Commerce DataSet.csv", encoding="ISO-8859-1")

In [5]:
df.head()
df.columns

Index(['rec_id', 'order_id', 'order_date', 'shipped_at', 'prod_sku',
       'prod_qty'],
      dtype='object')

In [6]:
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')
df['shipped_at'] = pd.to_datetime(df['shipped_at'], errors='coerce')

In [7]:
df['Month'] = df['order_date'].dt.to_period('M').astype(str)

In [8]:
np.random.seed(42)
price_map = {sku: np.random.randint(5, 100) for sku in df['prod_sku'].unique()}
df['UnitPrice'] = df['prod_sku'].map(price_map)
df['Sales'] = df['prod_qty'] * df['UnitPrice']


In [9]:
platforms = ['Mobile App', 'Website', 'Marketplace']
df['Platform'] = np.random.choice(platforms, size=len(df))

In [11]:
total_orders = df['order_id'].nunique()
total_qty = df['prod_qty'].sum()
total_sales = df['Sales'].sum()
avg_delay = (df['shipped_at'] - df['order_date']).dt.days.mean()

kpis = go.Figure()

kpis.add_trace(go.Indicator(
    mode="number",
    value=total_orders,
    title={"text": "Total Orders"},
    domain={'row': 0, 'column': 0}
))
kpis.add_trace(go.Indicator(
    mode="number",
    value=total_qty,
    title={"text": "Total Quantity"},
    domain={'row': 0, 'column': 1}
))
kpis.add_trace(go.Indicator(
    mode="number",
    value=total_sales,
    title={"text": "Total Revenue ($)"},
    domain={'row': 0, 'column': 2}
))
kpis.add_trace(go.Indicator(
    mode="number",
    value=avg_delay,
    title={"text": "Avg. Shipping Delay (days)"},
    domain={'row': 0, 'column': 3}
))

kpis.update_layout(
    grid={'rows': 1, 'columns': 4, 'pattern': "independent"},
    height=200
)
kpis.show()


In [12]:
monthly = df.groupby('Month').agg({'order_id':'nunique','Sales':'sum'}).reset_index()
fig1 = px.line(monthly, x='Month', y=['order_id','Sales'],
               labels={'value':'Count / Revenue','variable':'Metric'},
               title="Orders & Revenue Over Time")
fig1.show()

In [13]:
fig2 = px.pie(df, names='Platform', values='Sales', title="Revenue by Platform")
fig2.show()

In [14]:
top_products = df.groupby('prod_sku').agg({'prod_qty':'sum','Sales':'sum'}).reset_index()
top_products = top_products.sort_values('Sales', ascending=False).head(10)
fig3 = px.bar(top_products, x='prod_sku', y='Sales',
              title="Top 10 Products by Sales", text='Sales')
fig3.show()

In [15]:
df['delay_days'] = (df['shipped_at'] - df['order_date']).dt.days
fig4 = px.histogram(df, x='delay_days', nbins=20, title="Shipping Delay Distribution")
fig4.show()

In [16]:
df['Weekday'] = df['order_date'].dt.day_name()
weekly = df.groupby('Weekday').agg({'order_id':'nunique','Sales':'sum'}).reset_index()

In [17]:
weekday_order = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
weekly['Weekday'] = pd.Categorical(weekly['Weekday'], categories=weekday_order, ordered=True)
weekly = weekly.sort_values('Weekday')

fig5 = px.bar(weekly, x='Weekday', y='Sales', title="Revenue by Day of Week", text='Sales')
fig5.show()

In [18]:
df['Month_Name'] = df['order_date'].dt.month_name()
seasonal = df.groupby('Month_Name').agg({'order_id':'nunique','Sales':'sum'}).reset_index()

fig6 = px.line(seasonal, x='Month_Name', y='Sales',
               title="Seasonal Revenue Trends", markers=True)
fig6.show()

In [19]:
df['order_date'] = pd.to_datetime(df['order_date'])
df['shipped_at'] = pd.to_datetime(df['shipped_at'])


In [20]:
monthly_orders = df.groupby(df['order_date'].dt.to_period('M')).size()
peak_month = monthly_orders.idxmax().strftime("%B %Y")
peak_orders = monthly_orders.max()

In [21]:
top_skus = (
    df.groupby('prod_sku')['prod_qty']
    .sum()
    .sort_values(ascending=False)
    .head(5)
)


In [22]:
df['shipping_delay_days'] = (df['shipped_at'] - df['order_date']).dt.days
avg_delay = df['shipping_delay_days'].mean()


In [23]:
if 'Revenue' in df.columns:
    top_revenue_sku = (
        df.groupby('prod_sku')['Revenue']
        .sum()
        .sort_values(ascending=False)
        .head(1)
    ).index[0]
else:
    top_revenue_sku = None

In [25]:
print(f"- The busiest month was {peak_month} with {peak_orders} orders.")
print(f"- The top 5 SKUs by quantity sold are: {', '.join(top_skus.index)}.")
print(f"- Average shipping delay is {avg_delay:.1f} days.")
if top_revenue_sku:
    print(f"- The highest revenue-generating SKU was {top_revenue_sku}.")

- The busiest month was March 2015 with 8116 orders.
- The top 5 SKUs by quantity sold are: ROON_24465_06th Oct_TV_2nd_Duplicate, 8.9014E+12, ROON_24478_30th Oct_TV_AD_4th_Duplicate, ROON_24494_13th_Nov_TV_AD, ROON_244322_14Aug.
- Average shipping delay is 52.8 days.


# Insights

### Dataset Background
The dataset used in this project is the **E-Commerce Dataset - South Asia (100K Records)** from Kaggle.  
It contains **100,000 customer order records** from a large e-commerce retailer in South Asia.  
The dataset includes six columns:  
- `rec_id` and `order_id` as unique identifiers,  
- `order_date` and `shipped_at` for tracking order creation and fulfillment times,  
- `prod_sku` (product identifier), and  
- `prod_qty` (quantity ordered).  

This dataset provides insights into customer ordering behavior, product demand, and shipping performance in a typical e-commerce fulfillment center.

### Findings
- The busiest month was **March 2015**, with a total of **8,116 orders**, showing a strong seasonal peak.  
- The **top 5 SKUs** by quantity sold were:  
  1. ROON_24465_06th Oct_TV_2nd_Duplicate  
  2. 8.9014E+12  
  3. ROON_24478_30th Oct_TV_AD_4th_Duplicate  
  4. ROON_24494_13th_Nov_TV_AD  
  5. ROON_244322_14Aug  
  This indicates that sales were highly concentrated around a few products.  
- The **average shipping delay** was **52.8 days**, a significant lag that suggests operational inefficiencies affecting customer experience.

### Conclusion
The analysis highlights both **demand concentration** (few SKUs driving most sales) and **operational challenges** (long shipping delays).  
These findings suggest opportunities for improving **inventory management** and **fulfillment processes** in South Asian e-commerce.

